# DETOXIFICATION

In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from googletrans import Translator
from transformers import Trainer, TrainingArguments, T5Tokenizer, T5ForConditionalGeneration

device = "cuda" if torch.cuda.is_available() else "cpu"
device

c:\Users\afrai\Desktop\MAADM\NLP\nlp_detox\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'cpu'

## Lectura de datos

Vamos a leer el dataset de entrenamiento. Este dataset esta formado por 400 frases tóxicas por idioma en 9 idiomas diferentes y sus versiones no tóxicas. Formaremos un dataset conjunto que contenga para cada frase, su idioma de origen, la frase tóxica y la frase de-toxificada.

In [2]:
splits = {'en': 'data/en-00000-of-00001.parquet', 'ru': 'data/ru-00000-of-00001.parquet', 'uk': 'data/uk-00000-of-00001.parquet', 'de': 'data/de-00000-of-00001.parquet', 'es': 'data/es-00000-of-00001.parquet', 'am': 'data/am-00000-of-00001.parquet', 'zh': 'data/zh-00000-of-00001.parquet', 'ar': 'data/ar-00000-of-00001.parquet', 'hi': 'data/hi-00000-of-00001.parquet'}
df_en = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["en"])
df_uk = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["uk"])
df_ru = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["ru"])
df_de = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["de"])
df_es = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["es"])
df_am = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["am"])
df_zh = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["zh"])
df_ar = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["ar"])
df_hi = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["hi"])


In [4]:
df_total = pd.concat([df_en, df_uk, df_ru, df_de, df_es, df_am, df_zh, df_ar, df_hi], keys=['en','uk', 'ru', 'de', 'es', 'am', 'zh-cn', 'ar', 'hi']).reset_index(level=[0])

In [5]:
df_total.sample(3)

,level_0,toxic_sentence,neutral_sentence
8,uk,Кольорова печать на звичайному папері 10 гриве...,Кольорова печать на звичайному папері 10 гриве...
112,hi,वाह दोगले हरामी पत्रकार सही न्यूज़ नहीं चलाई ?,वाह पक्षपाती बुरे पत्रकार सही न्यूज़ नहीं चलाई ?
80,ar,يا متخلف الهدايه بيد الله يمكن ربي يهديهم باخر...,الهدايه بيد الله يمكن ربي يهديهم باخر لحظه…


### Train-test split

In [11]:
train, test = train_test_split(df_total, train_size=0.8, random_state=33)

X_train = list(train['toxic_sentence'])
y_train = list(train['neutral_sentence'])

X_test = list(test['toxic_sentence'])
y_train = list(test['neutral_sentence'])

## Pipeline de detoxificación

Nuestro modelo de detoxificación tendrá un pipeline que consistirá de los siguientes pasos:

1. **Traducción**: Traduciremos las frases del idioma original al inglés
2. **Detoxificación**: El modelo detoxificará la frase en inglés
3. **Retro-traducción**: La frase detoxificada será traducida del inglés al idioma original

### Traducción y retrotraducción

Usaremos la libreria de google translate para crear un modelo que traduzca un texto de un lenguaje a otro, pasados por argumentos.

In [6]:
from googletrans import Translator

translator = Translator()

In [7]:
def translate(text, source_lang, dest_lang):
    if source_lang == dest_lang:
        return text
    
    sol = translator.translate(text , src = source_lang, dest=dest_lang).text
    return sol

In [9]:
test_trad = df_total.sample(5).to_numpy()

for sentence in test_trad:
    print(f"Original sentence ({sentence[0]}): {sentence[1]}")
    translation = translate(sentence[1], sentence[0], "en")
    print(f"Transalation: {translation}")
    print("-"*50)

Original sentence (ru): не могут писад конституцию пускай не пишут эти пидарасв
Transalation: they can't write the constitution, don't let these fagots write it
--------------------------------------------------
Original sentence (ru): их. действительно. убили. вопрос кто. это те. стремилы. которые увласти. пидарасты. все. которые. сидят в думе. проститутки
Transalation: their. really. killed. the question is who. these are the ones. stirrups. who are in power. fagots. All. which. sit in the Duma. prostitutes
--------------------------------------------------
Original sentence (am): አንቺ የውሻ ልጅ የሚሰረቅ ያለው በኢትዮጽያ ስም የተመዘገበ የሕዝብ ንብረት ነው። መሀይም
Transalation: You puppy is stealing public property registered in the name of Ethiopia. Ignorant
--------------------------------------------------
Original sentence (de): Dummes Gelaber wegen Abschiebung in unsichere Staaten. Berlin, Manchester. und Nizza liegen auch in unsicheren Staaten. Also keine Hemmungen.
Transalation: Stupid talk about deporta

## Detoxificación

Ahora haremos un finetunning de un modelo de Huggingface para detoxificar el código.

In [ ]:
model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

num_training_layers = 2 # El número de capas que se entrenarán

# Congelar todos los parámetros del modelo
for param in model.parameters():
    param.requires_grad = False

# Descongelar las últimas n capas del decoder
for param in model.decoder.block[-num_training_layers:].parameters():
    param.requires_grad = True

# Mantener la capa de salida (`lm_head`) entrenable
for param in model.lm_head.parameters():
    param.requires_grad = True

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("ESTADÍSTICAS DEL ENTRENAMIENTO:")
print(f"Total de parámetros: {total_params:,}")
print(f"Parámetros entrenables: {trainable_params:,}")
print(f"Parámetros congelados: {total_params - trainable_params:,}")

c:\Users\afrai\Desktop\MAADM\NLP\nlp_detox\.venv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\afrai\.cache\huggingface\hub\models--google--flan-t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5

ESTADÍSTICAS DEL ENTRENAMIENTO:
Total de parámetros: 76,961,152
Parámetros entrenables: 22,744,064
Parámetros congelados: 54,217,088


#### Creación de los dataset

In [ ]:
class DetoxificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, target_encodings):
        self.encodings = encodings
        self.target_encodings = target_encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.target_encodings['input_ids'][idx].clone().detach()
        return item

In [ ]:
X_train_tokenized = tokenizer(X_train, truncation=True, padding=True, return_tensors="pt")
y_train_tokenized = tokenizer(y_train, truncation=True, padding=True, return_tensors="pt")

X_test_tokenized = tokenizer(X_test, truncation=True, padding=True, return_tensors="pt")
y_test_tokenized = tokenizer(y_train, truncation=True, padding=True, return_tensors="pt")

train_dataset = DetoxificationDataset(X_train_tokenized, y_train_tokenized)
val_dataset = DetoxificationDataset(X_test_tokenized, y_test_tokenized)

training_args = TrainingArguments(
    output_dir='./checkpoints',
    evaluation_strategy="no", 
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    learning_rate=2e-4,
    num_train_epochs=5,
    report_to=["none"],
    fp16=True, # acelerar entrenaminento 
)